In [16]:
#libraries for the neural network
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dropout, Input, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
#libraries for movie edition
from moviepy.editor import VideoFileClip
import os

## Building of the Neural Network using weights

In [17]:
#the three following cells are used to recreate the model for which the weights were calculated
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB1(include_top=False,input_tensor=Input(shape=(224, 224, 3)))

In [18]:
top_layers = efficientnet_model.output
top_layers = Flatten(name="flatten_top")(top_layers)
top_layers = Dense(1024, activation="ReLU",name="first_dense_top")(top_layers)
top_layers = Dropout(0.5, name="dropout_top")(top_layers)
top_layers = Dense(units=3, activation="softmax",name="linear_output")(top_layers)

In [19]:
model = Model(inputs=efficientnet_model.input, outputs=top_layers)

In [23]:
#loading the weights
cp_dir = "weights"
weights = tf.train.latest_checkpoint(cp_dir)
model.load_weights(weights)

## Setup for the analysis

In [24]:
video_file="79466_2019_V9.040EU_V9.039EU.mpg" # path of the video
video_clip = VideoFileClip(video_file)

In [25]:
nb_frame_per_sec=1
framing_rate = min(video_clip.fps, nb_frame_per_sec)
step = 1 / video_clip.fps if framing_rate == 0 else 1 / framing_rate

In [26]:
#creating a new directory to store framed images
filename, _ = os.path.splitext(video_file)
filename += "-framed"
if not os.path.isdir(filename):
	os.mkdir(filename)

## Analysis

In [35]:
ts_Fissure = []
ts_Racines = []
for current_duration in np.arange(0, video_clip.duration, step):
    frame_filename = os.path.join(filename, f"frame{current_duration}.jpg")
    # save the frame with the current duration
    video_clip.save_frame(frame_filename, current_duration)
    #predict the label
    img = tf.keras.utils.load_img(frame_filename, target_size=(224, 224))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = model.predict(img_array)
    #take the label the most probable
    pred=np.argmax(predictions[0])
    
    if pred==0 : # corresponds to cracks
        mins = np.floor(current_duration/60)
        secs = (current_duration/60 - mins)*60
        ts_Fissure.append([mins, secs])
    elif pred==1 : # corresponds to roots
        mins = np.floor(current_duration/60)
        secs = (current_duration/60 - mins)*60
        ts_Racines.append([mins, secs])


In [36]:
ts_Fissure=np.int_(np.floor(ts_Fissure))
ts_Racines=np.int_(np.floor(ts_Racines))

## Printing

In [37]:
print('Cracks found in time stamps:')
for timeStamps in ts_Fissure:
    print(str(timeStamps[0])+'\''+str(timeStamps[1]) + '\"')

print('\n')
print('Roots found in time stamps:')
for timeStamps in ts_Racines:
    print(str(timeStamps[0])+'\''+str(timeStamps[1]) + '\"')

Cracks found in time stamps:
0'3"
0'4"
0'5"
0'6"
0'7"
0'17"
0'18"
0'20"
0'21"
0'22"
0'23"
0'24"
0'25"
0'26"
0'27"
0'28"
0'29"
0'30"
0'31"
0'32"
0'33"
0'34"
0'35"
0'53"
0'57"
0'58"
0'59"
1'0"
1'0"
1'2"
1'3"
1'3"
1'4"
1'30"
1'30"
1'32"
1'33"
1'34"
1'34"
1'36"
1'37"
1'41"
1'42"
1'42"
1'44"
1'45"
1'46"
1'47"
1'48"
1'49"
1'49"
1'51"
1'52"
1'53"
1'53"
2'2"
2'4"
2'5"
2'6"
2'7"
2'7"
2'8"
2'10"
2'19"
2'21"
2'31"
2'32"
2'34"
2'57"
2'58"
2'59"
3'0"
3'0"
3'1"
3'2"
3'4"
3'7"
3'8"
3'9"
3'12"
3'13"
3'14"
3'15"
3'15"
3'16"
3'17"
3'19"
3'22"
3'23"
3'23"
3'29"
3'30"
3'30"
3'31"
3'36"
3'37"
3'42"
3'43"
3'44"
3'45"
3'46"
3'46"
3'47"
3'49"
3'50"
3'51"
3'52"
3'53"
3'53"
3'54"
3'55"
3'57"
3'58"
4'0"
4'0"
4'1"
4'2"
4'3"
4'4"
4'5"
4'6"
4'8"
4'9"
4'10"
4'24"
4'27"
4'28"
4'29"
4'30"
4'30"
4'31"
4'32"
4'33"
4'34"
4'35"
4'36"
4'38"
4'39"
4'40"
4'41"
4'42"
4'43"
4'44"
4'45"
4'46"
4'46"
4'47"
4'48"
4'49"
4'50"
4'51"
4'53"
4'54"
4'55"
4'56"
4'57"
4'58"
4'59"
5'0"
5'0"
5'1"
5'2"
5'3"
5'4"
5'5"
5'6"
5'8"
5'9"
5'10"
5'1